# TODO: Title

This notebook lists all the steps that you need to complete the complete this project. You will need to complete all the TODOs in this notebook as well as in the README and the two python scripts included with the starter code.


**TODO**: Give a helpful introduction to what this notebook is for. Remember that comments, explanations and good documentation make your project informative and professional.

**Note:** This notebook has a bunch of code and markdown cells with TODOs that you have to complete. These are meant to be helpful guidelines for you to finish your project while meeting the requirements in the project rubrics. Feel free to change the order of these the TODO's and use more than one TODO code cell to do all your tasks.

In [2]:
# TODO: Install any packages that you might need
# For instance, you will need the smdebug package
!pip install smdebug

/opt/conda/lib/python3.7/site-packages/secretstorage/dhcrypto.py:16: CryptographyDeprecationWarning: int_from_bytes is deprecated, use int.from_bytes instead
  from cryptography.utils import int_from_bytes
/opt/conda/lib/python3.7/site-packages/secretstorage/util.py:25: CryptographyDeprecationWarning: int_from_bytes is deprecated, use int.from_bytes instead
  from cryptography.utils import int_from_bytes
     |████████████████████████████████| 270 kB 30.3 MB/s            
     |████████████████████████████████| 83 kB 168 kB/s              
You should consider upgrading via the '/opt/conda/bin/python -m pip install --upgrade pip' command.


In [4]:
# TODO: Import any packages that you might need
# For instance you will need Boto3 and Sagemaker
import sagemaker
import boto3

## Dataset
TODO: Explain what dataset you are using for this project. Maybe even give a small overview of the classes, class distributions etc that can help anyone not familiar with the dataset get a better understand of it.

Dataset chosen: Dogs images.

In [ ]:
#TODO: Fetch and upload the data to AWS S3

# Command to download and unzip data
!wget https://s3-us-west-1.amazonaws.com/udacity-aind/dog-project/dogImages.zip
!unzip dogImages.zip

### Exploring and Explaining the data

In [9]:
#dir names = classes dog name
import os
folder = './dogImages/train'
train_folders = [name for name in os.listdir(folder) if os.path.isdir(os.path.join(folder, name))]

print(train_folders)

['114.Otterhound', '110.Norwegian_lundehund', '127.Silky_terrier', '091.Japanese_chin', '095.Kuvasz', '061.English_cocker_spaniel', '043.Canaan_dog', '131.Wirehaired_pointing_griffon', '124.Poodle', '087.Irish_terrier', '053.Cocker_spaniel', '064.English_toy_spaniel', '105.Neapolitan_mastiff', '032.Boston_terrier', '017.Bearded_collie', '094.Komondor', '003.Airedale_terrier', '084.Icelandic_sheepdog', '122.Pointer', '098.Leonberger', '125.Portuguese_water_dog', '009.American_water_spaniel', '058.Dandie_dinmont_terrier', '115.Papillon', '006.American_eskimo_dog', '041.Bullmastiff', '085.Irish_red_and_white_setter', '081.Greyhound', '004.Akita', '103.Mastiff', '044.Cane_corso', '118.Pembroke_welsh_corgi', '013.Australian_terrier', '117.Pekingese', '002.Afghan_hound', '025.Black_and_tan_coonhound', '129.Tibetan_mastiff', '093.Kerry_blue_terrier', '023.Bernese_mountain_dog', '070.German_pinscher', '052.Clumber_spaniel', '048.Chihuahua', '088.Irish_water_spaniel', '050.Chinese_shar-pei', '1

In [17]:
import pandas as pd
df = pd.Series(data = train_folders)
df.sample(10)

30             044.Cane_corso
16       003.Airedale_terrier
109    128.Smooth_fox_terrier
8                  124.Poodle
46               092.Keeshond
128               014.Basenji
94          067.Finnish_spitz
45     019.Bedlington_terrier
87        107.Norfolk_terrier
32     013.Australian_terrier
dtype: object

### Upload data to S3

In [4]:
#Using the sagemaker SDK grab the current region, execution role, and bucket.
import sagemaker
import boto3
ses = sagemaker.Session()

bucket = ses.default_bucket() ## TODO: fill in
print("Default Bucket: {}".format(bucket))

region = ses.boto_region_name ## TODO: fill in
print("AWS Region: {}".format(region))

role = sagemaker.get_execution_role() ## TODO: fill in
print("RoleArn: {}".format(role))

Default Bucket: sagemaker-us-east-1-513508352252
AWS Region: us-east-1
RoleArn: arn:aws:iam::513508352252:role/sagemaker_full_access_3


In [ ]:
## save train images
import os

os.environ["DEFAULT_S3_BUCKET"] = bucket
!aws s3 sync ./dogImages s3://${DEFAULT_S3_BUCKET}/dogImages/
print('DogImage complete ---------------------------!')

In [6]:
# confirm that data is in S3 bucket
empty_check = []
for obj in boto3.resource('s3').Bucket(bucket).objects.all():
    empty_check.append(obj.key)
    #print(obj.key)

assert len(empty_check) !=0, 'S3 bucket is empty.'
print('Upload complete!')

Upload complete!


## Hyperparameter Tuning
**TODO:** This is the part where you will finetune a pretrained model with hyperparameter tuning. Remember that you have to tune a minimum of two hyperparameters. However you are encouraged to tune more. You are also encouraged to explain why you chose to tune those particular hyperparameters and the ranges.

**Note:** You will need to use the `hpo.py` script to perform hyperparameter tuning.

Now as the next part of the project we will finetune ResNet model with hyperparameter tuning. We decided to fine tune four hyperparameters. The four hyper parameters fine tuned are as below: 1)Learning rate, which is also a continuous hyper parameter is fine tuned in (0.0001, 0.1)  2)eps which is a conitnuous hyper parameter is fine tuned in (1e-8, 1e-5)  3)Batch size is the categorical hyper parameter and is fine tuned in[32, 64, 128, 256, 512]

In [7]:
import sagemaker
sagemaker_session = sagemaker.Session()
from  sagemaker.pytorch import PyTorch
from sagemaker.tuner import (
    IntegerParameter,
    CategoricalParameter,
    ContinuousParameter,
    HyperparameterTuner,
)


In [20]:
#TODO: Declare your HP ranges, metrics etc.
hyperparameters = {"test-batch-size": "100", "n_gpus": 4}
hyperparameter_ranges = {
    "lr": ContinuousParameter(0.001, 0.1),
    "batch-size": CategoricalParameter([32, 64, 128, 256, 512]),
    "epochs": IntegerParameter(2,10)
}

objective_metric_name = "average test loss"
objective_type = "Minimize"
metric_definitions = [{"Name": "average test loss", "Regex": "Test set: Average loss: ([0-9\\.]+)"}]

In [22]:
#TODO: Create estimators for your HPs


# TODO: Your estimator here
estimator = PyTorch(
    entry_point = "hpo.py",
    base_job_name = "dog_image_classification_resnet18",
    role = role,
    instance_count = 1,
    instance_type = "ml.g4dn.xlarge", #"ml.m5.large"
    hyperparameters = hyperparameters,
    framework_version="1.4.0",
    py_version = "py3",
    output_path = "s3://sagemaker-us-east-1-513508352252/output/"
)

# TODO: Your HP tuner here
tuner = HyperparameterTuner(
    estimator,
    objective_metric_name,
    hyperparameter_ranges,
    metric_definitions,
    max_jobs = 4,
    max_parallel_jobs = 2,
    objective_type = objective_type
)

In [ ]:
# TODO: Fit your HP Tuner
input_data = 's3://sagemaker-us-east-1-513508352252/dogImages/'

tuner.fit({"training": input_data}, wait = True) # TODO: Remember to include your data channels

....................................................................................................................

In [ ]:
# TODO: Get the best estimators and the best HPs

best_estimator = tuner.best_estimator() #TODO


#Get the hyperparameters of the best trained model
best_hyperparameters = best_estimator.hyperparameters()
best_hyperparameters


## Model Profiling and Debugging
TODO: Using the best hyperparameters, create and finetune a new model

**Note:** You will need to use the `train_model.py` script to perform model profiling and debugging.

In [ ]:
# TODO: Set up debugging and profiling rules and hooks
from sagemaker.debugger import Rule, DebuggerHookConfig, rule_configs, CollectionConfig
from sagemaker.debugger import ProfilerRule, ProfilerConfig, FrameworkProfile

profiler_config = ProfilerConfig(
    system_monitor_interval_millis=500,
    framework_profile_params=FrameworkProfile(num_steps=10)
)

rules = [
    Rule.sagemaker(rule_configs.vanishing_gradient()),
    Rule.sagemaker(rule_configs.overfit()),
    Rule.sagemaker(rule_configs.overtraining()),
    Rule.sagemaker(rule_configs.poor_weight_initialization()),
    Rule.sagemaker(rule_configs.loss_not_decreasing()),
    ProfilerRule.sagemaker(rule_configs.LowGPUUtilization()),
    ProfilerRule.sagemaker(rule_configs.ProfilerReport()),
]

hook_config = DebuggerHookConfig(
    hook_parameters = {"train.save_interval": "10", "eval.save_interval": "5"},
    collection_configs = [CollectionConfig(name = "CrossEntropyLoss_output_0",
                                           parameters = {
                                               "include_regex": "CrossEntropyLoss_output_0",
                                               "train.save_interval": "10",
                                               "eval.save_interval": "1"})]
)

hyperparameters = { "test-batch-size": "100",
                   "epochs": 9, #best_hyperparameters['epochs'],
                   "batch-size": 256, #int(best_hyperparameters['batch-size'].replace('"', '')),
                   "lr": 0.087281525554365} #best_hyperparameters['lr']}
hyperparameters

In [ ]:
# TODO: Create and fit an estimator

estimator = PyTorch(
    entry_point = "train_model.py",
    base_job_name = "best-imgclassification",
    role = get_execution_role(),
    instance_count = 1,
    instance_type = "ml.g4dn.2xlarge",
    hyperparameters = hyperparameters,
    framework_version = "1.8",
    py_version = "py36",
    output_path = "s3://project-imgclassify/output-best/",
    rules = rules,
    profiler_config = profiler_config,
    debugger_hook_config = hook_config
)# TODO: Your estimator here

In [ ]:
# TODO: Plot a debugging output.

**TODO**: Is there some anomalous behaviour in your debugging output? If so, what is the error and how will you fix it?  
**TODO**: If not, suppose there was an error. What would that error look like and how would you have fixed it?

In [ ]:
# TODO: Display the profiler output

## Model Deploying

In [ ]:
# TODO: Deploy your model to an endpoint

predictor=estimator.deploy() # TODO: Add your deployment configuration like instance type and number of instances

In [ ]:
# TODO: Run an prediction on the endpoint

image = # TODO: Your code to load and preprocess image to send to endpoint for prediction
response = predictor.predict(image)

In [ ]:
# TODO: Remember to shutdown/delete your endpoint once your work is done
predictor.delete_endpoint()